In [1]:
import GPy
import gp_estimator
import numpy as np
from pathos.multiprocessing import ProcessingPool as Pool
import time

In [3]:
x = np.array([[0.0055, 0.    , 2.0611, 2.1779, 2.049 , 2.1829],
 [0.0162, 0.0008, 2.404 , 2.4205, 2.3808, 2.438 ],
 [0.0293, 0.0023, 3.0397, 2.9771, 3.0244, 2.9875],
 [0.0504, 0.0048, 3.6016, 3.5267, 3.6161, 3.5082],
 [0.0793, 0.0088, 3.9944, 3.9703, 4.0511, 3.9107],
 [0.1071, 0.0141, 4.2249, 4.2913, 4.3218, 4.1928],
 [0.12  , 0.0201, 4.3301, 4.4981, 4.4512, 4.3768],])
y = np.array([[ 0.0591],
 [ 0.0655],
 [ 0.0651],
 [ 0.0469],
 [ 0.0204],
 [-0.0319],
 [-0.0498],])
x_new = np.array([[ 0.029 ,  0.0304,  2.0851,  2.1765,  2.0631,  2.1932],
 [ 0.0247,  0.0317,  2.3887,  2.3709,  2.3523,  2.4034],
 [ 0.0228,  0.0328,  2.9894,  2.8823,  2.9627,  2.9058],
 [ 0.0274,  0.0342,  3.5229,  3.4013,  3.5312,  3.3905],
 [ 0.0411,  0.0362,  3.8938,  3.831 ,  3.951 ,  3.7719],
 [ 0.057 ,  0.0391,  4.1085,  4.1521,  4.2117,  4.0479],
 [ 0.0608,  0.0421,  4.2044,  4.3692,  4.3341,  4.2392],
 [ 0.0373,  0.044 ,  4.2223,  4.4921,  4.3478,  4.3671],
 [-0.0236,  0.0428,  4.1963,  4.5302,  4.2815,  4.4455],
 [-0.1229,  0.0367,  4.1489,  4.4901,  4.1599,  4.4795],
 [-0.2518,  0.0241,  4.09  ,  4.3761,  4.001 ,  4.4654],
 [ 0.033 ,  0.0273,  2.0986,  2.2043,  2.0896,  2.2101],])

In [2]:
x = np.array([[0.0055, 0.    ],
              [0.0162, 0.0008],
              [0.0293, 0.0023],
              [0.0504, 0.0048],
              [0.0793, 0.0088],
              [0.1071, 0.0141],
              [0.12  , 0.0201],])
y = np.array([[ 0.0591],
 [ 0.0655],
 [ 0.0651],
 [ 0.0469],
 [ 0.0204],
 [-0.0319],
 [-0.0498],])
x_new = np.array([[ 0.029 ,  0.0304],
                  [ 0.0247,  0.0317],
                  [ 0.0228,  0.0328],
                  [ 0.0274,  0.0342],
                  [ 0.0411,  0.0362],
                  [ 0.057 ,  0.0391],
                  [ 0.0608,  0.0421],
                  [ 0.0373,  0.044 ],
                  [-0.0236,  0.0428],
                  [-0.1229,  0.0367],
                  [-0.2518,  0.0241],
                  [ 0.033 ,  0.0273],])

In [4]:
GP = gp_estimator.GP_estimator()

In [5]:
def predict_parallel(opt = True):
    prediction_args =  [(x,y,x_new,0,opt),
                        (x,y,x_new,1,opt),
                        (x,y,x_new,2,opt),
                        (x,y,x_new,3,opt),
                        (x,y,x_new,4,opt),
                        (x,y,x_new,5,opt)]
                                    
    f = lambda x: GP.predict_accel(*x)
                    
                
    with Pool(processes=1) as pool:
        result = pool.map(f, prediction_args)


In [6]:
def predict_serial(opt = True):
    mean, var = GP.predict_accel(x, y, x_new, 0, opt)
    mean, var = GP.predict_accel(x, y, x_new, 1, opt)
    mean, var = GP.predict_accel(x, y, x_new, 2, True)
    mean, var = GP.predict_accel(x, y, x_new, 3, opt)
    mean, var = GP.predict_accel(x, y, x_new, 4, opt)
    mean, var = GP.predict_accel(x, y, x_new, 5, opt)

In [8]:
%timeit predict_serial(False)
%timeit predict_parallel(True)

15.6 ms ± 1.45 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
56.7 ms ± 5.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit mean, var = GP.predict_accel(x, y, x_new, 0, True)


6.12 ms ± 403 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
